In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
import pandas as pd
from src.inference import fetch_next_hour_predictions, load_batch_of_features_from_store
from src.plot_utils import plot_aggregated_time_series

current_date = pd.Timestamp.now(tz="Etc/UTC")

features = load_batch_of_features_from_store(current_date)

2025-05-27 00:19:30,845 INFO: Initializing external client
2025-05-27 00:19:30,846 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-27 00:19:33,660 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1233057
Fetching data from 2025-04-28 04:19:30.838603+00:00 to 2025-05-27 03:19:30.838603+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.75s) 


In [4]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-05-26 05:00:00
1,0,2,0,0,0,1,1,1,0,0,...,0,1,0,0,1,0,0,0,3,2025-05-26 05:00:00
2,3,5,3,9,7,4,3,1,2,3,...,3,7,10,9,1,9,2,0,4,2025-05-26 05:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,2025-05-26 05:00:00
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,6,2025-05-26 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,0,0,1,1,0,0,0,1,0,0,...,1,0,1,0,0,0,1,0,259,2025-05-26 05:00:00
251,0,2,0,5,3,4,2,2,2,1,...,7,7,3,6,1,3,3,0,260,2025-05-26 05:00:00
252,3,0,4,13,19,15,15,48,41,52,...,14,15,8,3,3,2,3,1,261,2025-05-26 05:00:00
253,25,75,143,144,130,109,91,79,79,87,...,15,19,10,8,3,1,2,1,262,2025-05-26 05:00:00


In [5]:
predictions = fetch_next_hour_predictions()

2025-05-27 00:19:52,723 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-27 00:19:52,736 INFO: Initializing external client
2025-05-27 00:19:52,737 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-27 00:19:54,711 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1233057
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.52s) 
Current UTC time: 2025-05-27 04:19:52.723281+00:00
Next hour: 2025-05-27 05:00:00+00:00
Found 255 records


In [6]:
predictions


,pickup_location_id,predicted_demand,pickup_hour
0,220,0.0,2025-05-27 05:00:00+00:00
1,118,0.0,2025-05-27 05:00:00+00:00
2,232,8.0,2025-05-27 05:00:00+00:00
3,93,1.0,2025-05-27 05:00:00+00:00
4,195,0.0,2025-05-27 05:00:00+00:00
...,...,...,...
250,197,2.0,2025-05-27 05:00:00+00:00
251,76,1.0,2025-05-27 05:00:00+00:00
252,255,9.0,2025-05-27 05:00:00+00:00
253,148,42.0,2025-05-27 05:00:00+00:00


In [7]:
top10 = (
    predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].to_list()
)

In [8]:
top10

[237, 132, 138, 186, 142, 163, 68, 238, 48, 249]

In [9]:
top10

[237, 132, 138, 186, 142, 163, 68, 238, 48, 249]

In [10]:
features["pickup_location_id"] == 22

0      False
1      False
2      False
3      False
4      False
       ...  
250    False
251    False
252    False
253    False
254    False
Name: pickup_location_id, Length: 255, dtype: bool

In [11]:
features_cp = features.copy()

In [12]:
from datetime import timedelta
features = features_cp[features_cp["pickup_location_id"] == 161]
prediction = predictions[predictions["pickup_location_id"] == 161]
time_series_columns = [
    col for col in features.columns if col.startswith("rides_t-")
]
time_series_values = [features[col].iloc[0] for col in time_series_columns] + prediction["predicted_demand"].to_list()

# Convert pickup_hour Series to single timestamp
pickup_hour = pd.Timestamp(features["pickup_hour"].iloc[0])

# Generate corresponding timestamps for the time series
time_series_dates = pd.date_range(
    start=pickup_hour - timedelta(hours=len(time_series_columns)),
    end=pickup_hour,
    freq="h",
)

# Create a DataFrame for the historical data
historical_df = pd.DataFrame(
    {"datetime": time_series_dates, "rides": time_series_values}
)

In [13]:
historical_df

,datetime,rides
0,2025-04-28 05:00:00,7.0
1,2025-04-28 06:00:00,35.0
2,2025-04-28 07:00:00,76.0
3,2025-04-28 08:00:00,163.0
4,2025-04-28 09:00:00,163.0
...,...,...
668,2025-05-26 01:00:00,20.0
669,2025-05-26 02:00:00,6.0
670,2025-05-26 03:00:00,8.0
671,2025-05-26 04:00:00,14.0
